In [1]:
# import dependencies

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import psycopg2
from sqlalchemy import create_engine

# may delete or move these these once model is more defined
#  moved down to split section-> from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [2]:
#create database connection
conn = psycopg2.connect(
    user = 'postgres',
    password = 'FinalProjectDB',
    host = 'finalprojectdb.chf41rwaufvy.us-east-1.rds.amazonaws.com',
    port = 5432,
    database = 'FinalProjectDB')

In [3]:
# import data and create dataframe
df_un = pd.read_sql_query("select * from \"Accidents\"", conn)
pd.set_option('display.expand_frame_repr', False)
df_un.head(30)

,CASENUM,STRATUM,REGION,URBANICITY,MONTH,YEAR,DAY_WEEK,HARM_EV,ALCOHOL,MAX_SEV,...,INT_HWY,WEATHER,WKDY_IM,EVENT1_IM,MANCOL_IM,RELJCT1_IM,RELJCT2_IM,LGTCON_IM,WEATHR_IM,ALCHL_IM
0,'202002121240,9,4,2,1,2020,4,12,9,0,...,0,10,4,12,6,0,2,1,10,2
1,'202002121829,8,3,1,1,2020,4,42,2,2,...,0,1,4,42,0,0,1,2,1,2
2,'202002121849,9,3,1,1,2020,4,12,9,0,...,0,1,4,12,1,0,3,1,1,2
3,'202002123484,10,4,2,1,2020,4,57,9,0,...,1,4,4,57,0,1,19,1,4,2
4,'202002123576,10,1,2,1,2020,4,1,9,0,...,0,4,4,1,0,0,1,2,4,2
5,'202002125146,8,2,2,1,2020,4,34,9,1,...,0,1,4,34,0,0,1,2,1,2
6,'202002125149,10,2,2,1,2020,6,11,9,9,...,0,98,6,11,0,0,1,2,10,2
7,'202002125582,7,3,2,1,2020,5,12,9,0,...,0,2,5,12,6,0,2,1,2,2
8,'202002125605,10,3,1,1,2020,4,1,2,0,...,0,1,4,1,0,0,1,1,1,2
9,'202002125637,10,3,2,1,2020,4,1,2,0,...,0,1,4,1,0,0,3,1,1,2


In [4]:
#filter dataframe to remove unknows categories
df = df_un[df_un['MAX_SEV'] < 5]
df.head(30)

,CASENUM,STRATUM,REGION,URBANICITY,MONTH,YEAR,DAY_WEEK,HARM_EV,ALCOHOL,MAX_SEV,...,INT_HWY,WEATHER,WKDY_IM,EVENT1_IM,MANCOL_IM,RELJCT1_IM,RELJCT2_IM,LGTCON_IM,WEATHR_IM,ALCHL_IM
0,'202002121240,9,4,2,1,2020,4,12,9,0,...,0,10,4,12,6,0,2,1,10,2
1,'202002121829,8,3,1,1,2020,4,42,2,2,...,0,1,4,42,0,0,1,2,1,2
2,'202002121849,9,3,1,1,2020,4,12,9,0,...,0,1,4,12,1,0,3,1,1,2
3,'202002123484,10,4,2,1,2020,4,57,9,0,...,1,4,4,57,0,1,19,1,4,2
4,'202002123576,10,1,2,1,2020,4,1,9,0,...,0,4,4,1,0,0,1,2,4,2
5,'202002125146,8,2,2,1,2020,4,34,9,1,...,0,1,4,34,0,0,1,2,1,2
7,'202002125582,7,3,2,1,2020,5,12,9,0,...,0,2,5,12,6,0,2,1,2,2
8,'202002125605,10,3,1,1,2020,4,1,2,0,...,0,1,4,1,0,0,1,1,1,2
9,'202002125637,10,3,2,1,2020,4,1,2,0,...,0,1,4,1,0,0,3,1,1,2
10,'202002125644,7,3,2,1,2020,5,52,2,3,...,1,10,5,52,0,0,1,1,10,2


In [5]:
# separate targets

y = df["MAX_SEV"]
y.tail(5)


54740    2
54741    1
54742    2
54743    0
54744    0
Name: MAX_SEV, dtype: int64

In [6]:
#close data source connection
conn.close()

In [7]:
#create resulting features dataframe
X_df = df[["REGION","URBANICITY","MONTH","YEAR","DAY_WEEK","HARM_EV","ALCOHOL","MAN_COLL","RELJCT1","RELJCT2","TYP_INT","WRK_ZONE","REL_ROAD","LGT_COND","INT_HWY","WEATHER"]]
X_df.tail(5)

,REGION,URBANICITY,MONTH,YEAR,DAY_WEEK,HARM_EV,ALCOHOL,MAN_COLL,RELJCT1,RELJCT2,TYP_INT,WRK_ZONE,REL_ROAD,LGT_COND,INT_HWY,WEATHER
54740,3,2,12,2020,4,52,2,0,0,1,1,0,4,1,0,10
54741,3,1,11,2020,5,11,2,0,0,1,1,0,1,2,0,1
54742,3,1,11,2020,3,34,9,0,0,1,1,0,4,1,0,1
54743,3,1,11,2020,2,12,2,6,1,2,2,0,1,2,0,1
54744,3,1,12,2020,2,12,2,6,0,8,1,0,1,8,0,1


In [8]:
# partition data into training and test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, 
                                                    y, random_state =1,test_size = .2)
X_train.shape

(42833, 16)

In [9]:
#Resample the training data to account for variations in the sampe
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [10]:
# create the model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100)

In [11]:
# train the model *can toggle between resampled data
classifier.fit(X_resampled, y_resampled)
#classifier.fit(X_train, y_train)

RandomForestClassifier()

In [12]:
# make preditions with the model
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,1,1
1,0,4
2,3,2
3,0,1
4,0,1
5,1,0
6,4,2
7,1,1
8,3,1
9,3,3


In [13]:
# export results to a csv file
df_results=pd.DataFrame(results)
df_results.to_csv('results_no_veh.csv')

In [14]:
# validate the model

In [15]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.4104024652161733


In [16]:
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[2885  856  541  367   74]
 [ 999  642  515  296   57]
 [ 627  407  524  291   69]
 [ 366  236  291  311   68]
 [  70   45   56   83   33]]


In [17]:
# export confusion matrix to a csv file
df_matrix=pd.DataFrame(matrix)
df_matrix.to_csv('matrix_no_veh.csv')

In [18]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.58      0.61      0.60      4723
           1       0.29      0.26      0.27      2509
           2       0.27      0.27      0.27      1918
           3       0.23      0.24      0.24      1272
           4       0.11      0.11      0.11       287

    accuracy                           0.41     10709
   macro avg       0.30      0.30      0.30     10709
weighted avg       0.41      0.41      0.41     10709

